In [ ]:
# needed imports
import numpy as np
import os
import cv2

# deep learning library
import keras
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, Dropout, BatchNormalization, UpSampling2D, concatenate
from keras.optimizers import Adam
from keras import Model

# plotting images
import matplotlib.pyplot as plt

#Accessing intermediate layers




import torch
import torch.nn as nn
from keras.applications import vgg16
from collections import namedtuple

In [ ]:
# numpy array containing all the classes
class_labels = np.array((33, 34, 35, 36, 38, 39, 40, 0))

In [ ]:
# #coding:utf8


# class Vgg16(torch.nn.Module):
#     def __init__(self):
#         super(Vgg16, self).__init__()
#         features = list(vgg16(pretrained = True).features)[:23]
        
#         # features的第3，8，15，22层分别是: relu1_2,relu2_2,relu3_3,relu4_3
#         self.features = nn.ModuleList(features).eval() 
#         print(self.features)
        
#     def forward(self, x):
#         results = []
#         for ii,model in enumerate(self.features):
#             x = model(x)
#             if ii in {3,8,15,22}:
#                 results.append(x)
#         vgg_outputs = namedtuple("VggOutputs", ['relu1_2', 'relu2_2', 'relu3_3', 'relu4_3'])
#         return vgg_outputs(*results)

In [ ]:
# vgg_16 = Vgg16()
# x_in = Input(shape=(128, 128, 3))
# c = vgg_16.forward(x_in)[1][1]
# c

In [ ]:
#coding:utf8
# THIS THING WORKS BUT WE DON'T KNOW HOW TO CONTINUE
from keras.applications import vgg16


class Vgg16():
    def __init__(self):
        self._vgg16 = vgg16.VGG16(weights='imagenet', include_top=False, input_shape = (None,None,3))
        self.features = [l for l in self._vgg16.layers]
        
    def forward(self, x):
        results = []
        for ii,layer in enumerate(self.features):
            x = layer(x)
            if ii in {5,9,13,17}:
                results.append(x)
        vgg_outputs = namedtuple("VggOutputs", ['relu_0','relu1_2', 'relu2_2', 'relu3_3'])
        return vgg_outputs(*results)

In [11]:
# since we are going to segment the image, the input should not have any dimension
x_in = Input(shape = (None,None,3))

vgg_16 = vgg16.VGG16(weights='imagenet', include_top=False,  input_tensor = x_in)

for l in vgg_16.layers:
    l.trainable = False


#-----------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------
# CONV 0

x = Conv2D(filters = 256,
           kernel_size = (3,3),
           kernel_initializer = 'he_uniform',
           padding = 'same')(vgg_16.layers[-1].output)

x = BatchNormalization()(x)

x = Activation(activation = 'relu')(x)

x = Dropout(0.2) (x)

# CONV + UPSAMPLING 1

x = UpSampling2D(size = (2,2 ))(x)

x = concatenate([x,vgg_16.get_layer("block5_conv3").output])

x = Conv2D(filters = 256,
           kernel_size = (3,3),
           kernel_initializer = 'he_uniform',
           padding = 'same')(x)

x = BatchNormalization()(x)

x = Activation(activation = 'relu')(x)

x = Dropout(0.2) (x)

# CONV + UPSAMPLING 2

x = UpSampling2D(size = (2, 2))(x)

x = concatenate([x,vgg_16.get_layer("block4_conv3").output])

x = Conv2D(filters = 128,
           kernel_size = (3,3),
           kernel_initializer = 'he_uniform',
           padding = 'same')(x)

x = BatchNormalization()(x)

x = Activation(activation = 'relu')(x)

x = Dropout(0.2) (x)

# CONV + UPSAMPLING 3

x = UpSampling2D(size = (2, 2))(x)

x = concatenate([x,vgg_16.get_layer("block3_conv3").output])

x = Conv2D(filters = 64,
           kernel_size = (3,3),
           kernel_initializer = 'he_uniform',
           padding = 'same')(x)

x = BatchNormalization()(x)

x = Activation(activation = 'relu')(x)

x = Dropout(0.2)(x)

# CONV + UPSAMPLING 4

x = UpSampling2D(size = (2, 2))(x)

#x = concatenate([x,vgg_16.get_layer("block2_conv3").output])

x = Conv2D(filters = 32,
           kernel_size = (3,3),
           kernel_initializer = 'he_uniform',
           padding = 'same')(x)

x = BatchNormalization()(x)

x = Activation(activation = 'relu')(x)

x = Dropout(0.2) (x)

# CONV + UPSAMPLING 5

x = UpSampling2D(size = (2, 2))(x)

#x = concatenate([x,vgg_16.get_layer("block1_conv3").output])

x = Conv2D(filters = 8,
           kernel_size = (3,3),
           kernel_initializer = 'he_uniform',
           padding = 'same')(x)

x = BatchNormalization()(x)

x = Activation(activation = 'relu')(x)

x = Dropout(0.2)(x)

#-----------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------

x_out = Activation(activation = 'softmax')(x)

vgg_16_new = Model(input = x_in, output = x_out)

vgg_16_new.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 6 1792        input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, None, None, 6 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, None, None, 6 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

/home/pabloleo96/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:117: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ac...)`


In [ ]:
# model name
model_name = 'best_model_vgg_8c.h5'
result_dir = './results'
vgg_16_new.load_weights(os.path.join(result_dir, model_name), by_name=True)

In [ ]:
test_dir = "./data/full_data/test/"

# obtain a list of files in the test directory
list_x_test = np.array(sorted(os.listdir(test_dir)))

n = 5
file_x_original = []
for i in range(n):
    file_x_original.append(np.array(cv2.imread(os.path.join(test_dir, list_x_test[800+i]), -1))/255)
    
file_x_original = np.array(file_x_original)
print(file_x_original[0].shape)
file_y_pred = vgg_16_new.predict(file_x_original[:,1200:2000,:])
print('file_y_pred.shape',file_y_pred.shape)

In [ ]:
#file_y_pred[file_y_pred > 0.3] = 1
#file_y_pred[file_y_pred < 0.3] = 0

# for i in range(n):
#     plt.figure(figsize = (15,12))
#     plt.subplot(2,3,1)
#     plt.imshow(file_x_original[i,1200:2000,:]); plt.title('original_color')
#     plt.subplfile_y_originalot(2,3,2)
#     plt.imshow(file_y_original[i,1200:2000,:]); plt.title('original_label')
#     plt.subplot(2,3,3)
#     plt.imshow(np.argmax(file_y_pred[i], axis=-1)); plt.title('predicted_label')
# plt.show()

# Post Processing - Sameera

In [ ]:
output_2D = np.array([np.argmax(image, axis=-1) for image in file_y_pred])
NN, HH, WW = output_2D.shape

print(output_2D.shape)

In [ ]:
for q in range(NN):
    print (list_x_test[q][:-4])

In [ ]:
## Pixel Count
import pandas as pd

tlabel = output_2D[0]
cls = np.unique(tlabel)
# print(cls)
# print(class_labels)
unique, counts = np.unique(tlabel, return_counts=True)

# d = dict(zip(unique, counts))
# class_labels = np.array((33, 34, 35, 36, 38, 39, 40, 0))
label_list = np.array((0,1,2,3,4,5,6,7))
label_map = dict(zip(label_list, class_labels))
print (label_list)
print (label_map)
print(unique, counts)
newList= []
for lb in range(len(cls)):
    newList.append(label_map[cls[lb]])
# print('newlist:',newList)
d = dict(zip(newList, counts))
print (d)

# classdict = {0:'others', 1:'rover', 17:'sky', 33:'car', 34:'motorbicycle', 35:'bicycle', 36:'person', 37:'rider', 38:'truck', 39:'bus', 40:'tricycle', 49:'road', 50:'siderwalk', 65:'traffic_cone'}
# df = pd.DataFrame.from_dict(d, orient='index').transpose()
# df.rename(columns=classdict, inplace=True)
# df

## To DO List
### ImageId,   LabelId, Confidence, Pixel, Count, EncodedPixels
### 001bbdb5c4f43bd4dc2b3e3a08b7202a, 33, 300, 1, 0 100|0 100|0 100|

1. Find Correct  ImageId

In [ ]:
# filelist_x_or[0][:-4]
test_dir = os.path.join("data/full_data/test")
test_file_paths = os.listdir(test_dir)
print(len(test_file_paths))


In [ ]:
ImageId = []
LabelId = []
Confidence = []
PixelCount = []
EncodedPixels = []

import time
start_time = time.time()
print ('WW', WW)
for image_id in range(1):
    image_id = 4
    ## Pixel Count
    tlabel = output_2D[image_id]
    plt.imshow(tlabel); plt.title('train_label')
    plt.show()
    print('image dim:', tlabel.shape)
    cls = np.unique(tlabel)
    unique, counts = np.unique(tlabel, return_counts=True)

    label_list = np.array((0,1,2,3,4,5,6,7))
    label_map = dict(zip(label_list, class_labels))
#     print (label_list)
#     print (label_map)
#     print(unique, counts)
    newList= []
    for lb in range(len(cls)):
        newList.append(label_map[cls[lb]])
    # print('newlist:',newList)
    d = dict(zip(newList, counts))
    print (d)
    
    print('\nimage_id', image_id)
    for class_val in range(1):
        class_val=5
        print('\nclass:', class_val)
        print('\label_map:', label_map[class_val])
        Harr, Warr = np.where(output_2D[image_id] == class_val)
        print('Harr',Harr)
        print('Warr',Warr)

        
        confidence = 0
        if len(file_y_pred[image_id,Harr,Warr,class_val])!=0:
            confidence = np.max(file_y_pred[image_id,Harr,Warr,class_val])
            print('confidence', confidence)
            row, count = np.unique(Harr, return_counts=True)# for each class

            Start = []
            mul1 = np.multiply(row, WW)
            for st in range(len(row)):
                ww2 = Warr[ np.where(Harr==row[st])[0][0] ]
                Start.append( mul1[st] + ww2 )
            
            encodedPix_str = '|'
            for tp in range(len(Start)):
                encodedPix_str = encodedPix_str + str(Start[tp])+' '+str(Start[tp] + count[tp])+'|'

            Image_id = list_x_test[800+image_id][:-4]
            ImageId.append(Image_id)
            LabelId.append(label_map[class_val])
            PixelCount.append(d[label_map[class_val]])
            Confidence.append(confidence)
            EncodedPixels.append(encodedPix_str)
            
    #     print('\nImageId:',ImageId)
    #     print('LabelId:',label_map[class_val])
    #     print('PixelCount:',d[label_map[class_val]])
    #     print('confidence:',confidence)
    #     print('EncodedPixels:')
    #     print(encodedPix_str)

print("\n\nEND--- %s seconds ---" % (time.time() - start_time))
# per each image - there will be 6 rows in the submission.csv
# make the csv
pd_columns = ['ImageId','LabelId','PixelCount','Confidence','EncodedPixels']

DF = pd.DataFrame(columns=pd_columns)

DF['ImageId'] = ImageId
DF['LabelId'] = LabelId
DF['PixelCount'] = PixelCount
DF['Confidence'] = Confidence
DF['EncodedPixels'] = EncodedPixels

print(DF)

#DF.to_csv('submit.csv', index=False)


In [ ]:
print(Warr)

In [ ]:

wer=20
print(row[wer])
QQQ = np.where(Harr==row[wer])
print(len(QQQ[0]))
print(QQQ[0])
print(Harr)
print(Warr[   np.where(Harr==row[wer])[0][0]    ])

In [ ]:
## Pixel Encoding

img_number = 0 # n from NN

#These var. will be created for each image (n)
encode_0 = []


confidance_dict = {}
for x in range(8):
    confidance_dict[x]=0

for h in range(1000):
    start = [None,None,None,None,None,None,None,None]
    count = [0,0,0,0,0,0,0,0]
    
    for w in range(3000,4000):
        
        # go though labels
        for lbl in range(len(label_list)-1):
            if output_2D[img_number][h][w]==lbl:#pixel with car label detected
                #calc. start pixel number for each class
                if start[lbl]==None:
                    start[lbl] = w
                else:
                    count[lbl] = count[lbl]+1 # count pixel belong to each class

                # get confidence level
                lbl_pos = np.argmax(file_y_pred[img_number][h][w])
                if (lbl_pos==lbl) and (file_y_pred[img_number][h][w][lbl]>confidance_dict[lbl]):
                    confidance_dict[lbl] = file_y_pred[img_number][h][w][lbl]
                
#     for lbl in range(len(label_list)):
    if start[0]!=None:
#         print ('h:',h,'  |',start[0]*h,' ',count[0],'|')
        encode_0.append([start[0]*h,count[0]])


In [ ]:
## Pixel Encoding +++++ BACKUP

img_number = 0 # n from NN

#These var. will be created for each image (n)
encode_0 = []
encode_1 = []
encode_2 = []
encode_3 = []
encode_4 = []
encode_5 = []
encode_6 = []
encode_7 = []

confidance_dict = {}
for x in range(8):
    confidance_dict[x]=0

for h in range(1000):
    start = [None,None,None,None,None,None,None,None]
    count = [0,0,0,0,0,0,0,0]
    
    for w in range(3000,4000):
        
        # go though labels
        for lbl in range(len(label_list)-1):
            if output_2D[img_number][h][w]==lbl:#pixel with car label detected
                #calc. start pixel number for each class
                if start[lbl]==None:
                    start[lbl] = w
                else:
                    count[lbl] = count[lbl]+1 # count pixel belong to each class

                # get confidence level
                lbl_pos = np.argmax(file_y_pred[img_number][h][w])
                if (lbl_pos==lbl) and (file_y_pred[img_number][h][w][lbl]>confidance_dict[lbl]):
                    confidance_dict[lbl] = file_y_pred[img_number][h][w][lbl]
                
#     for lbl in range(len(label_list)):
    if start[0]!=None:
#         print ('h:',h,'  |',start[0]*h,' ',count[0],'|')
        encode_1.append([start[0]*h,count[0]])





In [ ]:
print(encode)

print(confidance_dict)

# POst processing END

# loss function based on IoU

In [ ]:
import numpy as np
import keras
from keras.layers import *
import keras.backend as K
from keras.models import Model

In [ ]:
def compute_IoU(y_true, y_pred):
    # convert to numpy array for convenience and flatten
    pred = K.eval(y_pred)
    pred_f = pred.flatten()
    truth = K.eval(y_true)
    truth_f = truth.flatten()
    
    out_shape = pred.shape
        
    IoU = np.zeros(out_shape)
    
    lbls_in_pred = np.unique(pred_f)
    for pr_class in lbls_in_pred:
        # create mask of units that predicted pr_class
        mask_pr = (pred_f == pr_class)
        mask_tr = (truth_f == pr_class)
        
        # get intersection and union values for pr_class
        inter = np.sum(mask_pr == mask_tr)
        union = np.sum(mask_pr + mask_tr - inter)
        
        iou = (inter / union)
        
        # use value only on units that did the prediction
        iou_arr = mask_pr * iou
        
        # reshape and put in dictionary
        IoU += iou_arr.reshape(out_shape)
    
    IoU *= np.mean(IoU) 
    return IoU
    

In [ ]:
# pred = np.random.randint(0, 6, (300, 300))
# truth = np.random.randint(0, 6, (300, 300))

pred = np.random.randint(0, 2, (3, 3))
print(pred)
print()

truth = np.ones((3,3))

pred = K.variable(pred)
truth = K.variable(truth)

c = compute_IoU(truth, pred)
# i = np.unique(pred)[0]
# print(np.min(c), np.max(c))
print(c)


In [ ]:
import keras.applications.mobilenet as mn
c = mn.MobileNet()

c.summary()